In [ ]:
!pip install tashaphyne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.5/251.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 8.0 MB/s eta 0:00:00


In [ ]:
!pip install -U farasapy

# Importing libraries

In [ ]:
import spacy
import re
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import word_tokenize
from tashaphyne.stemming import ArabicLightStemmer
from nltk.stem import ISRIStemmer
import matplotlib.pyplot as plt
import seaborn as sns
nltk.download('punkt')
nltk.download('stopwords')
sp = spacy.load('en_core_web_sm')
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from snowballstemmer import stemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_test=pd.read_excel("/content/drive/MyDrive/NLP files-20240522T143610Z-001/NLP files/News_test (1).xlsx")
df_train=pd.read_excel("/content/drive/MyDrive/NLP files-20240522T143610Z-001/NLP files/News_train (1).xlsx")

# checking what needs to be preprocessed

In [ ]:
def unicode_markers(text):
  matches_1=re.findall(r'[\u202a-\u202e]', text)
  matches_2= re.findall(r'[\u2066-\u2069]', text)
  matches_3=re.findall(r'[\u202c\u202d]', text)
  return len(matches_1+matches_2+matches_3)


def english_words(text):
  matches=re.findall(r'\b[A-Za-z]+\b',text)
  return len(matches)


# Define counting functions
def diacritics(text):
     tshkil = re.compile(""" ّ    | # Tashdid
                       َ    | # Fatha
                       ً    | # Tanwin Fath
                       ُ    | # Damma
                       ٌ    | # Tanwin Damm
                       ِ    | # Kasra
                       ٍ    | # Tanwin Kasr
                       ْ    | # Sukun
                      ـ     # Tatwil/Kashida
                    """, re.VERBOSE)
     matches=re.findall(tshkil,text)
     return len(matches)

def normalizable_arabic_letters(text):
    matches_1 = re.findall(r'[إأآا]', text)
    matches_2 = re.findall(r'[ؤئ]', text)
    matches_3 = re.findall(r'[ة]', text)
    matches_4 = re.findall(r'[ى]', text)
    return len(matches_1) + len(matches_2) + len(matches_3) + len(matches_4)

def punctuation(text):
    matches=re.findall(r'[^\w\s]',text)  #remove punctiantion
    return len(matches)

def numbers(text):
    matches = re.findall(r'\d+', text)
    return len(matches)

def extra_spaces(text):
    matches = re.findall(r'\s{2,}', text)
    return len(matches)

def urls_or_emails(text):
    matches_1 = re.findall(r'http\S+|www\S+|https\S+', text)
    matches_2 = re.findall(r'\S+@\S+', text)
    return len(matches_1) + len(matches_2)
def emoji(text):
  emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
  matches=re.findall(emoji_pattern,text)
  return len(matches)

def check_the_preprocessing_needs(text):
  needs={
        'unicode':unicode_markers(text),
        'english words':english_words(text),
        'diacritics': diacritics(text),
        'normalizable_arabic_letters': normalizable_arabic_letters(text),
        'punctuation': punctuation(text),
        'numbers': numbers(text),
        'extra_spaces': extra_spaces(text),
        'urls_or_emails': urls_or_emails(text),
        'emojis':emoji(text)
    }
  return needs


In [ ]:
df_all=pd.concat([df_train,df_test],axis=0)

In [ ]:
needs=df_all['News'].apply(check_the_preprocessing_needs).tolist()
df_needs=pd.DataFrame(needs)
total_needs=df_needs.sum(axis=0)
total_needs

unicode                           30
english words                      8
diacritics                       629
normalizable_arabic_letters    67381
punctuation                     4380
numbers                          996
extra_spaces                     222
urls_or_emails                     0
emojis                             0
dtype: int64

# preprocessing the text

In [ ]:
def clean_text (text):
    #Cleaning
    text = re.sub(r'[\u202a-\u202e]', '', text)
    text = re.sub(r'[\u2066-\u2069]', '', text)
    text = re.sub(r'[\u202c\u202d]', '', text)
    text =re.sub(r'[^\w\s]',' ',text)  #remove punctiantion
    tshkil = re.compile(""" ّ    | # Tashdid
                       َ    | # Fatha
                       ً    | # Tanwin Fath
                       ُ    | # Damma
                       ٌ    | # Tanwin Damm
                       ِ    | # Kasra
                       ٍ    | # Tanwin Kasr
                       ْ    | # Sukun
                      ـ     # Tatwil/Kashida
                    """, re.VERBOSE)
    text = re.sub(tshkil, '', text)

    #Normalization
    text = re.sub(r'(.)\1+', r"\1\1", text) # Remove longation
    text = text.strip()
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub(r'[A-Za-z0-9]', r'', text)



    return text
def preprocess_text(text):
    #print(text)
    tokens = nltk.word_tokenize(text)
    tokens = [clean_text(token) for token in tokens]
    #print("normalize arabic:" ,tokens)

    stop_words = set(stopwords.words('arabic'))
    tokens = [token for token in tokens if token not in stop_words]
    #print("remove stop words:" ,tokens)

    ar_stemmer=stemmer("arabic")

    tokens = [ar_stemmer.stemWord(token) for token in tokens]


    #print("arabic stemmer:" ,tokens)

    processed_text = ' '.join(tokens)

    return processed_text


## trying different libraries for stemming

In [ ]:
test=df_train.copy().head()

In [ ]:
ar_stemmer1=ISRIStemmer()
ar_stemmer2=SnowballStemmer("arabic")# note that this is from nltk
ar_stemmer3=ArabicLightStemmer()
ar_stemmer4=stemmer("arabic") # this assem arabic light stemmer based on snowball


In [ ]:
# ISRIStemmer
for text in test['News']:
  print(text)
  tokens=nltk.word_tokenize(text)
  tokens=[clean_text(token) for token in tokens]
  tokens=[ar_stemmer1.stem(token) for token in tokens]
  print(tokens)

اشتباك الحريري-عون.. اتهامات لباسيل بالتمسك بالثلث المعطل للبقاء في الحكم
['شبك', 'حريري ع', '', 'تهم', 'اسل', 'تمس', 'ثلث', 'عطل', 'بقء', 'في', 'حكم']
عون: الحريري أصبح غريب الأطوار
['عون', '', 'حرر', 'اصبح', 'غرب', 'طور']
وزير الخارجية الأمريكي: ندرس سحب كامل قواتنا من أفغانستان بحلول مايو
['وزر', 'خرج', 'امر', '', 'درس', 'سحب', 'كمل', 'قوت', 'من', 'غنس', 'حلل', 'ايو']
أفغانستان.. استعدادات حثيثة لاجتماع تركيا وكابل تتهم طالبان بالتركيز على الحرب بدل السلام
['غنس', '', 'عدد', 'حثث', 'جمع', 'ترك', 'كبل', 'تهم', 'طلب', 'ركز', 'علي', 'حرب', 'بدل', 'سلم']
أندبندنت‮:‬ ‬مفاوضات ‬سرّية ‬‬كادت ‬تنقذ ‬القذافي
['اندبندنت', '', '', 'فاض', 'سر ', 'كدت', 'نقذ', 'قذف']


In [ ]:
# Snowball Stemmer
for text in test['News']:
  print(text)
  tokens=nltk.word_tokenize(text)
  tokens=[clean_text(token) for token in tokens]
  tokens=[ar_stemmer2.stem(token) for token in tokens]
  print(tokens)

اشتباك الحريري-عون.. اتهامات لباسيل بالتمسك بالثلث المعطل للبقاء في الحكم
['اشتب', 'حريري ع', '', 'اتهام', 'باسيل', 'تمسك', 'ثلث', 'معطل', 'بقاء', 'في', 'حكم']
عون: الحريري أصبح غريب الأطوار
['عون', '', 'حرير', 'اصبح', 'غريب', 'اطوار']
وزير الخارجية الأمريكي: ندرس سحب كامل قواتنا من أفغانستان بحلول مايو
['زير', 'خارجيه', 'امريك', '', 'ندرس', 'سحب', 'كامل', 'قوا', 'من', 'افغانست', 'حلول', 'ما']
أفغانستان.. استعدادات حثيثة لاجتماع تركيا وكابل تتهم طالبان بالتركيز على الحرب بدل السلام
['افغانست', '', 'استعداد', 'حثيث', 'اجتماع', 'ترك', 'كابل', 'تتهم', 'طالب', 'تركيز', 'عل', 'حرب', 'بدل', 'سلام']
أندبندنت‮:‬ ‬مفاوضات ‬سرّية ‬‬كادت ‬تنقذ ‬القذافي
['اندبندن', '', '', 'مفاوض', 'سر ', 'كاد', 'تنقذ', 'قذاف']


In [ ]:
# Arabic Light Stemmer
for text in test['News']:
  print(text)
  tokens=nltk.word_tokenize(text)
  tokens=[clean_text(token) for token in tokens]
  tokens=[ar_stemmer3.light_stem(token) for token in tokens]
  print(tokens)

اشتباك الحريري-عون.. اتهامات لباسيل بالتمسك بالثلث المعطل للبقاء في الحكم
['شتباك', 'حريري ع', '', 'اتهام', 'باسيل', 'تمسك', 'ثلث', 'معطل', 'بقاء', 'في', 'حكم']
عون: الحريري أصبح غريب الأطوار
['عو', '', 'حرير', 'صبح', 'غريب', 'اطوار']
وزير الخارجية الأمريكي: ندرس سحب كامل قواتنا من أفغانستان بحلول مايو
['زير', 'خارجيه', 'امريك', '', 'درس', 'سحب', 'امل', 'قوا', 'من', 'افغانس', 'حلول', 'ماي']
أفغانستان.. استعدادات حثيثة لاجتماع تركيا وكابل تتهم طالبان بالتركيز على الحرب بدل السلام
['افغانس', '', 'استعداد', 'حثيث', 'اجتماع', 'ركي', 'ابل', 'تهم', 'طالب', 'تركيز', 'على', 'حرب', 'دل', 'سلام']
أندبندنت‮:‬ ‬مفاوضات ‬سرّية ‬‬كادت ‬تنقذ ‬القذافي
['اندبندنت', '', '', 'مفاوض', 'سر ', 'ادت', 'نقذ', 'قذاف']


In [ ]:
# assem Arabic Light Stemmer
for text in test['News']:
  print(text)
  tokens=nltk.word_tokenize(text)
  tokens=[clean_text(token) for token in tokens]
  tokens=[ar_stemmer4.stemWord(token) for token in tokens]
  print(tokens)

اشتباك الحريري-عون.. اتهامات لباسيل بالتمسك بالثلث المعطل للبقاء في الحكم
['اشتب', 'حريري ع', '', 'اتهاما', 'لباسيل', 'تمسك', 'ثلث', 'معطل', 'بقاء', 'في', 'حكم']
عون: الحريري أصبح غريب الأطوار
['عون', '', 'حرير', 'اصبح', 'غريب', 'اطوار']
وزير الخارجية الأمريكي: ندرس سحب كامل قواتنا من أفغانستان بحلول مايو
['زير', 'خارجيه', 'امريك', '', 'ندرس', 'سحب', 'كامل', 'قوا', 'من', 'افغانست', 'حلول', 'مايو']
أفغانستان.. استعدادات حثيثة لاجتماع تركيا وكابل تتهم طالبان بالتركيز على الحرب بدل السلام
['افغانست', '', 'استعدادا', 'حثيث', 'لاجتماع', 'تركي', 'كابل', 'تتهم', 'طالب', 'تركيز', 'عل', 'حرب', 'بدل', 'سلام']
أندبندنت‮:‬ ‬مفاوضات ‬سرّية ‬‬كادت ‬تنقذ ‬القذافي
['اندبندن', '', '', 'مفاوضا', 'سر ', 'كاد', 'تنقذ', 'قذاف']


In [ ]:
tokens=nltk.word_tokenize(test)

TypeError: expected string or bytes-like object

In [ ]:
test

,Type,News
0,politics,اشتباك الحريري-عون.. اتهامات لباسيل بالتمسك با...
1,politics,عون: الحريري أصبح غريب الأطوار
2,politics,وزير الخارجية الأمريكي: ندرس سحب كامل قواتنا م...
3,politics,أفغانستان.. استعدادات حثيثة لاجتماع تركيا وكاب...
4,politics,أندبندنت‮:‬ ‬مفاوضات ‬سرّية ‬‬كادت ‬تنقذ ‬القذافي
processed_iris,NaN,NaN
